## Segmenting and Clustering Neighborhoods
----------------------------------

### Step 1

In [2]:
#!conda install -y lxml
#!conda install -y  -c conda-forge geocoder 
#!conda install -y  -c conda-forge folium
import pandas as pd
import lxml
import geocoder
import folium
print(lxml.__file__)
print(folium.__file__)


/home/anaconda/juyter/6.0.3/lib/python3.8/site-packages/lxml/__init__.py
/home/anaconda/juyter/6.0.3/lib/python3.8/site-packages/folium/__init__.py


In [3]:
#Data URL
data_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(data_url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [4]:
#Read to panda dataframe
df = pd.read_html(data_url)
print(len(df))
df[0].keys()

3


Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [6]:
#Get the first table from the wiki page and print the first ten rows
neighborhood_table=df[0]
#Change column names 
neighborhood_table.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
neighborhood_table.head(3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


In [7]:
#Drop rows where Borough is Not assigned
neighborhood_table.drop(neighborhood_table[neighborhood_table.Borough=='Not assigned'].index, inplace=True)

In [8]:
#Check the new dataframe
print(len(neighborhood_table))
neighborhood_table.reset_index(drop=True,inplace=True)
neighborhood_table.head(3)

103


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
neighborhood_table[neighborhood_table.Neighbourhood=='Not assigned'].index

Int64Index([], dtype='int64')

Quetions: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
**There are no such row so not taking any actions**

In [10]:
neighborhood_table.shape

(103, 3)

------------------------------


### Step 2

In [11]:
import geocoder # import geocoder
g = geocoder.google('Toronto, Ontario')
print(g.latlng)
g

None


<[REQUEST_DENIED] Google - Geocode [empty]>

In [10]:
#Test geocoder 
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print("latitude=",latitude)
#print("longitude=",longitude)

In [11]:
#goecoder fails, using the cvs
#coordinate_file="https://cocl.us/Geospatial_data"
#!wget -O 'coordinate.csv' coordinate_file https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
print('Data downloaded!')

Data downloaded!


In [12]:
coordinates = pd.read_csv("coordinate.csv") 
coordinates.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [13]:
coordinates.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
coordinates.set_index('PostalCode', inplace=True)
coordinates.head(10)

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [29]:
pc1="M1B"
c_lati=coordinates.loc[pc1][0]
c_long=coordinates.loc[pc1][1]

#print(coordinates.loc["M1B"][0])
print("Longitude=",c_long," Latitude=",c_lati)



Longitude= -79.19435340000001  Latitude= 43.806686299999996


In [30]:
#neighborhood_table.drop(columns=['Longitude='], inplace=True)
neighborhood_table['Longitude'] = 0
neighborhood_table['Latitude'] = 0
neighborhood_table.head(10)

,PostalCode,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,Parkwoods,0,0
1,M4A,North York,Victoria Village,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",0,0
6,M1B,Scarborough,"Malvern, Rouge",0,0
7,M3B,North York,Don Mills,0,0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0,0


In [16]:
#neighborhood_table.loc[neighborhood_table['PostalCode'] == "M7A",'Longitude'] =1
#neighborhood_table.head(10)
#Test set value

In [31]:
for pc in (neighborhood_table["PostalCode"]):
    c_long=coordinates.loc[pc][1]
    c_lati=coordinates.loc[pc][0]
    neighborhood_table.loc[neighborhood_table['PostalCode'] == pc,'Longitude'] = c_long
    neighborhood_table.loc[neighborhood_table['PostalCode'] == pc,'Latitude'] = c_lati
    #print(pc,"  Longitude=",c_long," Latitude=",c_lati)
    
neighborhood_table.head(10)

,PostalCode,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,Parkwoods,-79.329656,43.753259
1,M4A,North York,Victoria Village,-79.315572,43.725882
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360636,43.654260
3,M6A,North York,"Lawrence Manor, Lawrence Heights",-79.464763,43.718518
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-79.389494,43.662301
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",-79.532242,43.667856
6,M1B,Scarborough,"Malvern, Rouge",-79.194353,43.806686
7,M3B,North York,Don Mills,-79.352188,43.745906
8,M4B,East York,"Parkview Hill, Woodbine Gardens",-79.309937,43.706397
9,M5B,Downtown Toronto,"Garden District, Ryerson",-79.378937,43.657162


In [32]:
tpc="M9V"
neighborhood_table.loc[neighborhood_table['PostalCode'] == tpc] 


,PostalCode,Borough,Neighbourhood,Longitude,Latitude
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",-79.588437,43.739416


---------------------

### Step3

In [33]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhood_table['Borough'].unique()),
        neighborhood_table.shape[0]
    )
)
#neighborhood_table['Neighbourhood']

The dataframe has 10 boroughs and 103 neighborhoods.


In [36]:
# create map of New York using latitude and longitude values

#Center at Etobicoke
latitude  = "43.739416"
longitude= "-79.588437"

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_table['Latitude'], neighborhood_table['Longitude'], neighborhood_table['Borough'], neighborhood_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork